# vaccine-doses-on-hand

By [Sean Greene](https://www.latimes.com/people/sean-greene)

Downloads the number of vaccine doses on hand from a Tableau dashboard published by the California Department of Public Health.

## Import

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

Import dependencies.

In [72]:
import os
import re
import pytz
import json
import requests
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup

## Scrape

Set the URL

In [8]:
host = "https://public.tableau.com"

In [9]:
path = "/views/COVID-19VaccineProviderDashboardPublic/PublicVaccineProviderDashboard"

In [10]:
url = f"{host}{path}"

Set the custom variables we need to manipulate the dashboard.

In [11]:
config = dict(
    sheet_id="Main Vaccine Data Check (6)",
    value_index=3,
    value_key="aliasIndices",
    type_index=2,
    type_key="aliasIndices",
    label_index=1,
    label_key="aliasIndices",
)

Request the page

In [12]:
response = requests.get(url, params={":embed": "y", ":showVizHome": "no"})

Parse the HTML

In [18]:
soup = BeautifulSoup(response.text, "html.parser")

Zero in on the part of the page with the URL where we can get the data. It's in JSON format

In [21]:
json_string = soup.find("textarea", {"id": "tsConfigContainer"}).text

Parse it.

In [22]:
context = json.loads(json_string)

Pull out the URL

In [23]:
data_url = (
    f'{host}{context["vizql_root"]}/bootstrapSession/sessions/{context["sessionid"]}'
)

Then download the raw data, clean it up, and turn it into usable dictionaries.

In [25]:
response = requests.post(data_url, data={"sheet_id": config["sheet_id"]})

In [26]:
raw_text = response.text

In [27]:
json_pieces = [json.loads(d) for d in re.split("\d{2,10};(?={.+})", raw_text) if len(d)]

In [28]:
root = next(d for d in json_pieces if "secondaryInfo" in d)

In [29]:
data = root["secondaryInfo"]["presModelMap"]

Build our value lookup.

In [31]:
value_columns = data["dataDictionary"]["presModelHolder"]["genDataDictionaryPresModel"][
    "dataSegments"
]["0"]["dataColumns"]

In [32]:
lookup = {d["dataType"]: d["dataValues"] for d in value_columns}

Download the embed so we can scrape it and find the VizQL root ID to build our query.

In [33]:
response = requests.get(url, params={":embed": "y", ":showVizHome": "no"})

In [34]:
soup = BeautifulSoup(response.text, "html.parser")

In [35]:
context = json.loads(soup.find("textarea", {"id": "tsConfigContainer"}).text)

In [37]:
data_url = (
    f'{host}{context["vizql_root"]}/bootstrapSession/sessions/{context["sessionid"]}'
)

Then download the raw data, clean it up, and turn it into usable dictionaries.

In [38]:
response = requests.post(data_url, data={"sheet_id": config["sheet_id"]})

In [39]:
raw_text = response.text

In [40]:
json_pieces = [json.loads(d) for d in re.split("\d{2,10};(?={.+})", raw_text) if len(d)]

In [41]:
root = next(d for d in json_pieces if "secondaryInfo" in d)

In [42]:
data = root["secondaryInfo"]["presModelMap"]

Build our value lookup.

In [43]:
value_columns = data["dataDictionary"]["presModelHolder"]["genDataDictionaryPresModel"][
    "dataSegments"
]["0"]["dataColumns"]

In [44]:
lookup = {d["dataType"]: d["dataValues"] for d in value_columns}

Pull out the columns of indexes so we can run them against our lookup.

In [45]:
pres_model_map = data["vizData"]["presModelHolder"]["genPresModelMapPresModel"][
    "presModelMap"
]

In [47]:
columns = pres_model_map[config["sheet_id"]]["presModelHolder"]["genVizDataPresModel"][
    "paneColumnsData"
]["paneColumnsList"][0]["vizPaneColumns"]

Using our variables from above, pull out the lists of indexes we need.

In [49]:
labels_column = columns[config["label_index"]][config["label_key"]]

In [50]:
types_column = columns[config["type_index"]][config["type_key"]]

In [51]:
values_column = columns[config["value_index"]][config["value_key"]]

In [52]:
values_column_b = []
for val in values_column:
    values_column_b.append(abs(val) - 1)

Run each one through our lookup.

In [53]:
labels = [lookup["cstring"][label] for label in labels_column]

In [54]:
types = [lookup["cstring"][type] for type in types_column]

In [55]:
values_start = len(labels) + 4

In [56]:
values_end = len(labels)

In [57]:
values = [lookup["cstring"][value] for value in values_column_b]

In [58]:
try:
    assert len(labels) == len(values)
except AssertionError:
    raise AssertionError("Labels and values list do not add up")

`zip` and convert them to a `dict` so they are key/value'ed.

In [59]:
data = [
    {"agency": label, "doses_on_hand": value}
    for label, value in (sorted(zip(labels, values), key=lambda d: d[0]))
]

## Transform

Convert into a pandas DataFrame

In [66]:
df = pd.DataFrame(data)

Convert strings to integers

In [68]:
def safeint(s):
    """
    Convert the provided string to an integer. Return it.
    """
    s = s.replace(",", "")
    return int(s)

In [69]:
df["doses_on_hand"] = df["doses_on_hand"].apply(safeint)

## Export

In [62]:
tz = pytz.timezone("America/Los_Angeles")

In [65]:
today = datetime.now(tz).date()

In [75]:
data_dir = os.path.join(os.path.abspath(""), "data")

In [83]:
df.sort_values("agency").to_csv(os.path.join(data_dir, f"{today}.csv"), index=False)